In [ ]:
import kfp 
from kfp import dsl 
import kfp.components as comp
import os 

In [ ]:
@dsl.component
def download_from_redshift(a, b):
    import boto3 
    import pandas as pd
    import awswrangler as wr
    import os
    import psycopg2
    from sklearn.tree import DecisionTreeRegressor



    # with psycopg2 
    conn = psycopg2.connect(
    host="cdo-redshift-cluster-dev.c2nuq0toqpda.us-east-1.redshift.amazonaws.com",
    user="collibrain_dataclassification_rs_user",
    port=5439,
    password="8vC!ZmPuNF4eu#R4G5#V",
    dbname= "master")
    #cur = conn.cursor() # create a cursor for executing queries

    df = pd.read_sql_query('SELECT * FROM collibrain_dataclassification.labels', conn)
    df = df.dropna()
    
    # create a simple model since kfp doesn't seem to like being ran just for an etl workload.. 
    #regressor = DecisionTreeRegressor(random_state=42, max_depth=5)
    #regressor.fit(X, y)

    
    return df.output # kubeflow pipelines is acting weird if I just output this part above alone for some reason
    

In [ ]:
# execute the above function 
#download_red = kfp.components.create_component_from_func(download_from_redshift(schema="collibrain_dataclassification", dbname="master", table="labels"), base_image = "python:3.8", packages_to_install = ["boto3", "pandas", "awswrangler", "psycopg2"])

In [ ]:
@dsl.pipeline(
    name = "red etl 2",
    description = "Pull data from redshift and show it locally."
)
def red_pipeline(a: float, b: float) -> float:
    download_red = download_from_redshift()
    


In [ ]:
complr = kfp.compiler.Compiler()
complr.compile(red_pipeline, package_path = "red_etl.yaml")


In [ ]:
with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

client = kfp.Client(
    host='http://ml-pipeline.kubeflow.svc.cluster.local:8888',
    existing_token=TOKEN)
client.create_experiment("redshift_etl_test")


In [ ]:
client.create_run_from_pipeline_package(pipeline_file = 'red_etl.yaml', arguments={'a': 2.0}, experiment_name = "test")